# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4811, uptime 0:00:16
mini_beamline                    RUNNING   pid 4812, uptime 0:00:16
random_walk                      RUNNING   pid 4813, uptime 0:00:16
random_walk_horiz                RUNNING   pid 4814, uptime 0:00:16
random_walk_vert                 RUNNING   pid 4815, uptime 0:00:16
simple                           RUNNING   pid 4816, uptime 0:00:16
thermo_sim                       RUNNING   pid 4817, uptime 0:00:16
trigger_with_pc                  RUNNING   pid 4818, uptime 0:00:16


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-14 20:39:19
Persistent Unique Scan ID: '1849d4ae-5c9c-4eeb-b54b-0a417c0e2fe3'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:39:19.6 |     -1.000 |     102424 |


|         2 | 20:39:20.1 |     -0.500 |      99778 |


|         3 | 20:39:20.6 |      0.000 |      96671 |


|         4 | 20:39:21.1 |      0.500 |      94434 |


|         5 | 20:39:21.6 |      1.000 |      95109 |
+-----------+------------+------------+------------+
generator scan ['1849d4ae'] (scan num: 1)





('1849d4ae-5c9c-4eeb-b54b-0a417c0e2fe3',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/1849d4ae-5c9c-4eeb-b54b-0a417c0e2fe3-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-14 20:39:22
Persistent Unique Scan ID: 'b72dd12e-b693-4e46-b170-2dd065d6a8d5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:39:22.1 |          -1 | 1085126678 |


|         2 | 20:39:22.3 |          -0 | 1098050067 |


|         3 | 20:39:22.4 |           0 | 1104190770 |
|         4 | 20:39:22.5 |           0 | 1111405989 |


|         5 | 20:39:22.6 |           1 | 1116605084 |


+-----------+------------+-------------+------------+
generator scan ['b72dd12e'] (scan num: 2)





('b72dd12e-b693-4e46-b170-2dd065d6a8d5',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-14 20:39:23
Persistent Unique Scan ID: '958494cf-d953-4cac-a80f-05d062f67a35'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:39:23.2 | 1131527523 |


|         2 | 20:39:24.1 | 1096911715 |


|         3 | 20:39:25.1 | 1053430948 |


...

|        28 | 20:39:50.1 | 1085217700 |


|        29 | 20:39:51.1 | 1131669993 |


|        30 | 20:39:52.1 | 1094770435 |


+-----------+------------+------------+
generator count ['958494cf'] (scan num: 3)





('958494cf-d953-4cac-a80f-05d062f67a35',)